In [1]:
import os
import numpy as np
import pandas as pd
from IPython.display import display

import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.applications.efficientnet as efn
from sklearn.model_selection import GroupKFold

## Helper functions

## Variables and configurations

In [2]:
def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy


def build_decoder(with_labels=True, target_size=(256, 256), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels=None, bsize=32, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)
    
    return dset

In [3]:
strategy = auto_select_accelerator()

debug=True

COMPETITION_NAME = "ranzcr-clip-catheter-line-classification"
BATCH_SIZE = 8 if debug else strategy.num_replicas_in_sync * 16
n_folds=10
if not debug:
    from kaggle_datasets import KaggleDatasets
    GCS_DS_PATH = KaggleDatasets().get_gcs_path(COMPETITION_NAME)

Running on 1 replicas


## Preparing dataset

### Loading and preprocess CSVs

In [4]:
load_dir = "../input/ranzcr-clip-catheter-line-classification" if debug else f"/kaggle/input/{COMPETITION_NAME}"
df = pd.read_csv(f"{load_dir}/train.csv")

# paths = load_dir + "train/" + df['StudyInstanceUID'] + '.jpg'
paths = f"{load_dir}/train/"+df["StudyInstanceUID"]+".jpg" if debug \
    else f"{GCS_DS_PATH}/train/" + df['StudyInstanceUID'] + '.jpg'

sub_df = pd.read_csv(f"{load_dir}/sample_submission.csv")

# test_paths = load_dir + "test/" + sub_df['StudyInstanceUID'] + '.jpg'
test_paths = f"{load_dir}/train/"+df["StudyInstanceUID"]+".jpg" if debug \
    else f"{GCS_DS_PATH}/test/" + sub_df['StudyInstanceUID'] + '.jpg'

# Get the multi-labels
label_cols = sub_df.columns[1:]
labels = df[label_cols].values

In [5]:
def get_fold(train):
    fold=train.copy()
    splitter=GroupKFold(n_splits=n_folds)
    for n,(train_idx,val_idx) in enumerate(splitter.split(train,groups=train["PatientID"])):
        fold.loc[val_idx,"folds"]=n
    fold["folds"]=fold["folds"].astype(int)
    return fold

fold=get_fold(df)

In [10]:
target_fold=0

train_idx=(fold["folds"]!=target_fold)
val_idx=(fold["folds"]==target_fold)

train_paths=paths[train_idx]
valid_paths=paths[val_idx]

train_labels=labels[train_idx]
valid_labels=labels[val_idx]

# データの処理

In [11]:
# Build the tensorflow datasets
IMSIZES = (224, 240, 260, 300, 380, 456, 528, 600)
im_size = IMSIZES[7]

decoder = build_decoder(with_labels=True, target_size=(im_size, im_size))
test_decoder = build_decoder(with_labels=False, target_size=(im_size, im_size))

train_dataset = build_dataset(
    train_paths, train_labels, bsize=BATCH_SIZE, decode_fn=decoder
)

valid_dataset = build_dataset(
    valid_paths, valid_labels, bsize=BATCH_SIZE, decode_fn=decoder,
    repeat=False, shuffle=False, augment=False
)

## Modeling

In [14]:
n_labels = labels.shape[1]
def create_model(drop_connect_rate):  
    with strategy.scope():
        model = tf.keras.Sequential([
            efn.EfficientNetB7(
                input_shape=(im_size, im_size, 3),
                weights='imagenet',
                include_top=False,
                drop_connect_rate=drop_connect_rate), ##
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(n_labels, activation='sigmoid')
        ])
        model.compile(
            optimizer='adam',
            loss='binary_crossentropy',
            metrics=[tf.keras.metrics.AUC(multi_label=True,name="auc")])
    return model

In [15]:
import optuna

lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_auc", patience=3, min_lr=1e-6, mode='max')
checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5',
    save_best_only=True, monitor='val_auc', mode='max')
steps_per_epoch = train_paths.shape[0] // BATCH_SIZE

def objective(trial):
    keras.backend.clear_session() # clear session and release memory
    
    #最適化するパラメータの設定
    drop_connect_rate=trial.suggest_uniform("drop_connect_rate", 0.0, 0.4)         
    model = create_model(drop_connect_rate=drop_connect_rate)

        
    history = model.fit(
        train_dataset, 
        epochs=10,
        verbose=1,
        callbacks= [lr_reducer],
        steps_per_epoch=steps_per_epoch,
        validation_data=valid_dataset)

    auc=model.evaluate(valid_dataset)
    
    return auc

In [16]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

[I 2021-03-11 23:10:49,107] A new study created in memory with name: no-name-872da57a-b0b9-4346-afe1-9eb6fac4ec4d
Epoch 1/10


In [ ]:
result_df=study.trials_dataframe()
result_df.to_csv('efficientnetB7_optuna_history.csv')